In [ ]:
# imports
from Dynamic_only import *
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import CubicSpline, PchipInterpolator 
import time
import ast
import os
import shutil
from tqdm.notebook import tqdm

In [ ]:
key_intervals = pd.read_csv("../key_intervals.csv")
pattern_dates = pd.read_csv("../patterns_dates.csv", parse_dates=[0,1,2,3,4,5], date_format="%d/%m/%Y")

In [ ]:
key_intervals

In [ ]:
# Replication IDs mapped to their time interval
replication_ids = {'0500': 16686964, '0515': 16686990, '0530': 16686994, '0545': 16687067, '0600': 16687000, '0615': 16687004, '0630': 16687068,
 '0645': 16687010, '0700': 16687014, '0715': 16687018, '0730': 16687026, '0745': 16687028, '0800': 16687030, '0815': 16687048, '0830': 16687050,
 '0845': 16687053, '0900': 16687055, '0915': 16687057, '0930': 16687059, '0945': 16687061}

# time intervals mapped to the corresponding time interval
time_intervals = {index: key for index, key in enumerate(replication_ids.keys())}
time_intervals.update({-1:'0445'})

In [ ]:
def mono_cubic_interp(hook_pts, data, length):
    """
    Returns interpolated list of values 
    
    param hook_pts: List containing the hook points of a dataset to be interpolated
    param data: List containing the values at the given hook points
    param length: Length of required dataset
    """
    
    function = PchipInterpolator(hook_pts, data)
    x_new = np.array(range(length))
    interpolated_data_mono = function(x_new)
    return interpolated_data_mono


def output_od(vector_od):
    lookup = pd.read_csv("OD lookup.csv")
    lookup['count'] = vector_od
    lookup = lookup.pivot(index='id', columns='Destination')[['count']]
    return lookup['count']

In [ ]:
def interepolate_ODs():
    hookpts = pattern_key_intervals
    filenames = os.listdir("../Simulation ODs/")[1:-1]
    od_series = vectorize_od(pd.read_csv(f"../Simulation ODs/{filenames[0]}", index_col = "id"))
    for i in range(1, len(filenames)):
        new = vectorize_od(pd.read_csv(f"../Simulation ODs/{filenames[i]}", index_col = "id"))
        new = new.rename({0:i}, axis = 1)
        od_series = pd.concat([od_series, new], axis = 1)
    np_od = od_series.to_numpy()
    total_od = od_series.sum(axis = 1)
    non_zero_indices = [i for i in range(len(total_od)) if total_od[i] != 0 ]
    non_zero = [total_od[i] for i in non_zero_indices]
    
    for index in non_zero_indices:    
        data = np_od[index]
        inter_data = [data[i] for i in range(len(data)) if i in hookpts]
        new_data = mono_cubic_interp(hookpts, inter_data, 20)
        np_od[index] = new_data

    for file_index in range(20):
        output = output_od(np_od[:, file_index]).applymap(lambda x: 0 if x < 0 or np.isnan(x) else round(x))
        output.to_csv("../Simulation ODs/" + filenames[file_index])

In [ ]:
# declare constants
MAX_ASSIGNMENT_ITERATIONS = 5
GRADIENT_DESCENT_ITERATIONS = 8
OBJECTS_FILE = "sections_list.txt"
# DATE = str(pattern_dates[f'pattern_{pattern_id}'][0].date())
loss_evaluation = [] # list of loss function values at each iteration

In [ ]:
for pattern_id in range(9,10):
    pattern_key_intervals = ast.literal_eval(key_intervals.iloc[pattern_id - 4, 1])
    if not os.path.exists(f"../Output ODs/{pattern_id}/"):
        os.mkdir(f"../Output ODs/{pattern_id}/")
    for date_id in range(5,6):
        DATE = pattern_dates[f'pattern_{pattern_id}'][date_id]
        if str(DATE) != "NaT":
            DATE = str(DATE.date())
            if not os.path.exists(f"../Output ODs/{pattern_id}/{DATE}/"):
                os.mkdir(f"../Output ODs/{pattern_id}/{DATE}/")
            for outer_iterations in tqdm(range(MAX_ASSIGNMENT_ITERATIONS), leave = False):
                for interval in pattern_key_intervals:
                    time_interval = time_intervals[interval] # 24-hour time interval
                    run_sim(replication_ids[time_interval], time_interval) # Run simulation (should output a file "assignment_time_interval.txt" at the current location
                    assignment_filepath = f"assignment/assignment.txt"
                    aimsun_assignment = read_aimsun_assignment(assignment_filepath) # Generate assignment dataframe 
                    dynamic_matrices = create_dynamic_assignments(aimsun_assignment, OBJECTS_FILE, time_intervals, interval) # create dynamic assignment matrices
                    for inner_iterations in tqdm(range(GRADIENT_DESCENT_ITERATIONS), leave = False):
                        # loss_evaluation.append(objective_function(dynamic_matrices, time_intervals, interval, pattern_id, DATE))
                        dynamic_ods = get_dynamic_od(time_intervals, interval)
                        interval_gradients = gradient_interval(dynamic_matrices, time_intervals, interval, pattern_id, DATE)
                        step_length = step_len_interval(dynamic_ods, dynamic_matrices, interval_gradients, time_intervals, interval, pattern_id, DATE)
                        dynamic_ods[1] = dynamic_ods[1]*(1 - interval_gradients[1]*step_length[1][0])
                        update_ods(dynamic_ods[1], time_interval)
                interepolate_ODs()
            # Move adjusted ODs to the output folder
            for filenames in os.listdir("../Simulation ODs/"):
                os.rename(f"../Simulation ODs/{filenames}", f"../Output ODs/{pattern_id}/{DATE}/{filenames}")

            # Copy prior ODs to the simulation ODs folder
            for filenames in os.listdir(f"../Pattern ODs/{pattern_id}/"):
                shutil.copy(f"../Pattern ODs/{pattern_id}/{filenames}", "../Simulation ODs/")
            print(f"{DATE} completed for Pattern {pattern_id}.")

In [ ]:
DATE = "2021-06-02"
pattern_id = 9

In [ ]:
for pattern_id in range(5,10):
    for date_id in tqdm(range(10), leave = False):
        DATE = pattern_dates[f'pattern_{pattern_id}'][date_id]
        if str(DATE) != "NaT":
            DATE = str(DATE.date())
            if not os.path.exists(f"../Estimated flows/{pattern_id}/{DATE}/"):
                os.mkdir(f"../Estimated flows/{pattern_id}/{DATE}/")
            for filename in os.listdir(f"../Output ODs/{pattern_id}/{DATE}/"):
                shutil.copy(f"../Output ODs/{pattern_id}/{DATE}/{filename}", "../Simulation ODs/")
            for interval in tqdm(range(20), leave = False):
                time_interval = time_intervals[interval]
                run_sim(replication_ids[time_interval], time_interval) # Run simulation (should output a file "assignment_time_interval.txt" at the current location
                assignment_filepath = f"assignment/assignment.txt"
                aimsun_assignment = read_aimsun_assignment(assignment_filepath) # Generate assignment dataframe 
                dynamic_matrices = create_dynamic_assignments(aimsun_assignment, OBJECTS_FILE, time_intervals, interval)
                flows = estimate_dynamic_flow(dynamic_matrices, time_intervals, interval, 1)
                flows.to_csv(f"../Estimated flows/{pattern_id}/{DATE}/{time_intervals[interval]}.csv", index = False)
            print(f"{DATE} for {pattern_id} completed.")